<a href="https://colab.research.google.com/github/JulienDbrt/BGS-Invoice/blob/main/JSON_Brut_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install openai mistralai python-dotenv

# Skipped :
64060
63257
63155
63820
63155


## OpenAI

In [ ]:
# Extraction de "Pages", nettoyage de la données et envoie dans OpenAI API (le modèle varie selon le nombre de tokens estimé)

import os
import re
import csv
import glob
import json
import openai
import chardet
from tqdm import tqdm
from google.colab import userdata
from pydantic import BaseModel

openai.api_key = userdata.get('OPENAI_API_KEY')

directory_path = '/content/drive/MyDrive/EXPORT_3EME_WEB 4'
output_file_path = '/content/json_final_output_openai.csv'

class Result(BaseModel):
    vendorEmail: str
    uo_2: str
    invoiceNumber: str
    vendorCode: str
    commitmentCode: str
    title: str
    dueDate: str
    documentType: str
    vendorSiret: str
    projectCode: str
    ttc: str
    tiers: str
    ht: str
    dateDocument: str
    subChapter: str
    commitment: str
    project: str
    invoiceType: str

def estimate_token_count(text):
    """Estimate the number of tokens based on the text length.
    Rough estimation assuming an average token length, including spaces."""
    return len(text) / 5

def extract_data_json(json_file_path):
    """Extract and process data from a JSON file."""
    try:
        with open(json_file_path, 'rb') as file:
            raw_data = file.read()
            if not raw_data:
                print(f"Warning: The file {json_file_path} is empty.")
                return None
            encoding = chardet.detect(raw_data)['encoding'] or 'utf-8'
            json_data = json.loads(raw_data.decode(encoding))
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from the file {json_file_path}: {e}")
        return None

    pages_data = json_data.get('pages', [])
    sentences = []

    for page in pages_data:
        if page.get('$type') == 'PageContent':
            for text_zone in page.get('Items', []):
                if text_zone.get('$type') == 'TextZone':
                    for line in text_zone.get('Ln', []):
                        for item in line.get('Items', []):
                            if item.get('$type') == 'Word':
                                sentences.append(item.get('Value', ''))

    sentence = ' '.join(sentences)
    file_name = os.path.basename(json_file_path)
    return {'sentence': sentence, 'file_name': file_name}

def process_json_files(directory_path, output_file_path):
    json_files = glob.glob(f"{directory_path}/*.json")
    results = []
    print(f"Processing {len(json_files)} files...")

    for file_path in tqdm(json_files, desc="Progress"):
        print(f"Processing file: {file_path}")
        invoice_data = extract_data_json(file_path)
        if invoice_data is None:
            continue

        sentence = invoice_data.get('sentence', '')
        token_count = estimate_token_count(sentence)
        if token_count > 32000:
            max_length = 32000 * 5
            invoice_data['sentence'] = sentence[:max_length]

        document_details = invoice_data

        token_count = estimate_token_count(document_details)
        print(f"Estimated token count for {file_path}: {token_count}")

        model = 'gpt-4-0125-preview' if token_count > 15500 else 'gpt-3.5-turbo-0125'

        if token_count > 31000:
            print(f"{file_path} skipped due to token limit")
            continue

        prompt = [
            ChatMessage(role="system", content='Génère une sortie JSON avec les clés suivantes et leurs valeurs respectives à partir des informations que je te donne. Si la clé est absente, le champ est rempli avec une chaîne vide ou apparaît comme « NaN » (Not a Number) dans le jeu de données : "vendor Email" : adresse e-mail du fournisseur. "numeroPiece" : Contient la référence (Réf.) du document. S’il s’agit d’une situation de travaux, ajoute à cette référence le numéro de situation après un slash. "typeDocument" : Spécifie le type de document en fonction de la typologie suivante : (09.01 - Factures / 09.04 - Factures développement (avant promesse) / 09.07 - Factures marketing / 09.08 - Factures travaux et prorata / 09.21 - Honoraires juridiques / 09.22 - Honoraires prescripteurs / 09.31 - Situations travaux MOE BPCC / 09.32 - Situations travaux MOE externe. »). "tiersSiret" : Un numéro SIRET français unique à chaque entreprise française. "codeProjet" : Représente un code unique pour le projet associé à la facture ; composé de 14 chiffres : les 9 chiffres du Siren et 5 chiffres propres à l’établissement. "ttc" : Le montant total à payer, toutes taxes comprises (TTC). "tiers" : Le nom du tiers ou du fournisseur. Vigilance sour le taux de TVA appliqué car il y a de forte variabilité, tu dois te Baer sur le total TTC. "ht" : Le montant total du document au format hors taxes (HT). "dateDocument" : La date d’emission du document. "dueDate" : Indique la date de l échéance du document. Elle est fixée à 30 jours pour une facture, 45 ou 60 pour les engagements de travaux. En fonction de la date de facture, elle sera payée en milieu de mois (le 15) ou ou fin du mois (30 ou 31) suivant la date d’émission de la facture. Il faut compter 15 jours de plus pour les situations de travaux (1 mois 15 jours et maximum 2 mois). "typeFacture" : Indique le type de la facture (facture, situation de travaux ou avoir).'),
            ChatMessage(role="user", content=f"Here are the document details: \n{document_details}, create a JSON output")
        ]

        completion = client.chat.completions.create(model=model, response_format={ "type": "json_object" }, messages=prompt)
        llm_output = completion.choices[0].message.content
        invoice_data["LLM_output"] = llm_output
        results.append(invoice_data)

    with open(output_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = [
            "sentence", "file_name", "LLM_output"
        ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for item in results:
            writer.writerow(item)

process_json_files(directory_path, output_file_path)
